In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from scipy.stats import *
import sqlite3
import time
from collections import Counter
# next command ensures that plots appear inside the notebook
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns  # also improves the look of plots
sns.set()
plt.rcParams['figure.figsize'] = 10, 5  # default hor./vert. size of plots, in inches
plt.rcParams['lines.markeredgewidth'] = 1  # to fix issue with seaborn box plots; needed after import seaborn

In [2]:
df=pd.read_csv(r"C:\Users\Gebruiker\Downloads\BPI_Challenge_2012.xes.gz_UNPACKED (1).csv")


In [3]:
df

,Unnamed: 0,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
1,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
2,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
3,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
4,4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
...,...,...,...,...,...,...,...,...
262195,262195,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000
262196,262196,112.0,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000
262197,262197,11169.0,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000
262198,262198,11169.0,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000


In [4]:
df=df.dropna()


In [5]:
df


,Unnamed: 0,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
1,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
2,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
3,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
5,5,10862.0,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
...,...,...,...,...,...,...,...,...
262195,262195,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000
262196,262196,112.0,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000
262197,262197,11169.0,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000
262198,262198,11169.0,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000


In [6]:
list_of_events=df["concept:name"].unique()

In [7]:
list_of_events

array(['A_SUBMITTED', 'A_PARTLYSUBMITTED', 'A_PREACCEPTED',
       'W_Completeren aanvraag', 'A_ACCEPTED', 'O_SELECTED',
       'A_FINALIZED', 'O_CREATED', 'O_SENT', 'W_Nabellen offertes',
       'O_SENT_BACK', 'W_Valideren aanvraag', 'A_REGISTERED',
       'A_APPROVED', 'O_ACCEPTED', 'A_ACTIVATED', 'O_CANCELLED',
       'W_Wijzigen contractgegevens', 'A_DECLINED', 'A_CANCELLED',
       'W_Afhandelen leads', 'O_DECLINED',
       'W_Nabellen incomplete dossiers', 'W_Beoordelen fraude'],
      dtype=object)

In [8]:
df["next_common_event"]=df["concept:name"]
df

<ipython-input-8-58b765b1124d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["next_common_event"]=df["concept:name"]


,Unnamed: 0,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,next_common_event
0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_SUBMITTED
1,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PARTLYSUBMITTED
2,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PREACCEPTED
3,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,W_Completeren aanvraag
5,5,10862.0,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_ACCEPTED
...,...,...,...,...,...,...,...,...,...
262195,262195,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,A_PARTLYSUBMITTED
262196,262196,112.0,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads
262197,262197,11169.0,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads
262198,262198,11169.0,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,A_DECLINED


SyntaxError: unexpected EOF while parsing (<ipython-input-9-0b62700fd9d4>, line 5)

In [12]:
case = 'blib'
first_item = []
for index, row in df.iterrows():
    if case != row['case:concept:name']:
        case = row['case:concept:name']
        task = row['concept:name']
        first_item.append(task)
print(Counter(first_item))

Counter({'A_SUBMITTED': 13087})


In [13]:
def nth_most_common(w, n):
    sort = dict(sorted(Counter(w).items(), key=lambda item: item[1]))
    return str(list(sort)[-n])

In [14]:
def get_max(task):
    previous = 'blibs'
    empty = []
    case = 'blib'
    for index, row in df.iterrows():
        t = row['concept:name']
        c = row['case:concept:name']
        if (case == c) & (previous == task):
            empty.append(t)
        case = c
        previous = t
    return nth_most_common(empty, 1)

In [15]:
get_max('A_ACCEPTED')

'O_SELECTED'

In [16]:
caselist = list(df['concept:name'].unique())

In [17]:
a = {}
for value in caselist:
    a[value] = get_max(value)
a

{'A_SUBMITTED': 'A_PARTLYSUBMITTED',
 'A_PARTLYSUBMITTED': 'A_PREACCEPTED',
 'A_PREACCEPTED': 'W_Completeren aanvraag',
 'W_Completeren aanvraag': 'W_Completeren aanvraag',
 'A_ACCEPTED': 'O_SELECTED',
 'O_SELECTED': 'O_CREATED',
 'A_FINALIZED': 'O_CREATED',
 'O_CREATED': 'O_SENT',
 'O_SENT': 'W_Nabellen offertes',
 'W_Nabellen offertes': 'W_Nabellen offertes',
 'O_SENT_BACK': 'W_Valideren aanvraag',
 'W_Valideren aanvraag': 'W_Valideren aanvraag',
 'A_REGISTERED': 'A_ACTIVATED',
 'A_APPROVED': 'A_ACTIVATED',
 'O_ACCEPTED': 'A_REGISTERED',
 'A_ACTIVATED': 'W_Valideren aanvraag',
 'O_CANCELLED': 'O_SELECTED',
 'W_Wijzigen contractgegevens': 'W_Wijzigen contractgegevens',
 'A_DECLINED': 'W_Afhandelen leads',
 'A_CANCELLED': 'O_CANCELLED',
 'W_Afhandelen leads': 'W_Afhandelen leads',
 'O_DECLINED': 'A_DECLINED',
 'W_Nabellen incomplete dossiers': 'W_Nabellen incomplete dossiers',
 'W_Beoordelen fraude': 'W_Beoordelen fraude'}

In [19]:
import operator
import pickle

In [20]:
with open('next_common_event.pkl', 'wb') as f:
    pickle.dump(a, f)

In [21]:
with open('next_common_event.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [22]:
loaded_dict

{'A_SUBMITTED': 'A_PARTLYSUBMITTED',
 'A_PARTLYSUBMITTED': 'A_PREACCEPTED',
 'A_PREACCEPTED': 'W_Completeren aanvraag',
 'W_Completeren aanvraag': 'W_Completeren aanvraag',
 'A_ACCEPTED': 'O_SELECTED',
 'O_SELECTED': 'O_CREATED',
 'A_FINALIZED': 'O_CREATED',
 'O_CREATED': 'O_SENT',
 'O_SENT': 'W_Nabellen offertes',
 'W_Nabellen offertes': 'W_Nabellen offertes',
 'O_SENT_BACK': 'W_Valideren aanvraag',
 'W_Valideren aanvraag': 'W_Valideren aanvraag',
 'A_REGISTERED': 'A_ACTIVATED',
 'A_APPROVED': 'A_ACTIVATED',
 'O_ACCEPTED': 'A_REGISTERED',
 'A_ACTIVATED': 'W_Valideren aanvraag',
 'O_CANCELLED': 'O_SELECTED',
 'W_Wijzigen contractgegevens': 'W_Wijzigen contractgegevens',
 'A_DECLINED': 'W_Afhandelen leads',
 'A_CANCELLED': 'O_CANCELLED',
 'W_Afhandelen leads': 'W_Afhandelen leads',
 'O_DECLINED': 'A_DECLINED',
 'W_Nabellen incomplete dossiers': 'W_Nabellen incomplete dossiers',
 'W_Beoordelen fraude': 'W_Beoordelen fraude'}

In [24]:
list_of_events=df["concept:name"].unique()
list_of_events

array(['A_SUBMITTED', 'A_PARTLYSUBMITTED', 'A_PREACCEPTED',
       'W_Completeren aanvraag', 'A_ACCEPTED', 'O_SELECTED',
       'A_FINALIZED', 'O_CREATED', 'O_SENT', 'W_Nabellen offertes',
       'O_SENT_BACK', 'W_Valideren aanvraag', 'A_REGISTERED',
       'A_APPROVED', 'O_ACCEPTED', 'A_ACTIVATED', 'O_CANCELLED',
       'W_Wijzigen contractgegevens', 'A_DECLINED', 'A_CANCELLED',
       'W_Afhandelen leads', 'O_DECLINED',
       'W_Nabellen incomplete dossiers', 'W_Beoordelen fraude'],
      dtype=object)

In [25]:
df["time_till_next"]=df["time:timestamp"]

<ipython-input-25-5c411fa9f0cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time_till_next"]=df["time:timestamp"]


In [26]:
df

,Unnamed: 0,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,next_common_event,time_till_next
0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00
1,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00
2,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00
3,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00
5,5,10862.0,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00
...,...,...,...,...,...,...,...,...,...,...
262195,262195,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00
262196,262196,112.0,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00
262197,262197,11169.0,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00
262198,262198,11169.0,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,A_DECLINED,2012-03-01 09:27:37.118000+01:00


In [28]:
df_copy["time:timestamp"]= pd.to_datetime(df["time:timestamp"])

NameError: name 'df_copy' is not defined

In [29]:
df["time:timestamp"]= pd.to_datetime(df["time:timestamp"])

<ipython-input-29-0f9b8fb3268c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time:timestamp"]= pd.to_datetime(df["time:timestamp"])


In [30]:
df["case:REG_DATE"]= pd.to_datetime(df["case:REG_DATE"])

<ipython-input-30-80b2202a3cd9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["case:REG_DATE"]= pd.to_datetime(df["case:REG_DATE"])


In [31]:
df["time_till_next"]=pd.to_datetime(df["time_till_next"])

<ipython-input-31-bde2636cbe44>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time_till_next"]=pd.to_datetime(df["time_till_next"])


In [32]:
df

,Unnamed: 0,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,next_common_event,time_till_next
0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00
1,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00
2,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00
3,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00
5,5,10862.0,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00
...,...,...,...,...,...,...,...,...,...,...
262195,262195,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00
262196,262196,112.0,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00
262197,262197,11169.0,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00
262198,262198,11169.0,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,A_DECLINED,2012-03-01 09:27:37.118000+01:00


In [33]:
c=0
a=df.iloc[1,5]
for j in list_of_events:
    for i in range (1,244189):
        if df_.iloc[i,3]==j: 
            a=a+(df.iloc[i+1]["case:REG_DATE"]-df.iloc[i]["case:REG_DATE"])
            c=c+1
    print(j,a/c)

NameError: name 'df_copy' is not defined

In [35]:
df.iloc[0,3]

'A_SUBMITTED'

In [36]:
df["time_till_next"]=df["time:timestamp"]-df["case:REG_DATE"]

<ipython-input-36-11f473f61be8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time_till_next"]=df["time:timestamp"]-df["case:REG_DATE"]


In [37]:
df


,Unnamed: 0,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,next_common_event,time_till_next
0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_SUBMITTED,0 days 00:00:00
1,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PARTLYSUBMITTED,0 days 00:00:00.334000
2,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PREACCEPTED,0 days 00:00:53.360000
3,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,W_Completeren aanvraag,0 days 00:00:54.329000
5,5,10862.0,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_ACCEPTED,0 days 11:03:58.762000
...,...,...,...,...,...,...,...,...,...,...
262195,262195,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,A_PARTLYSUBMITTED,0 days 00:00:00.624000
262196,262196,112.0,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads,0 days 00:00:44.488000
262197,262197,11169.0,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads,0 days 09:35:29.937000
262198,262198,11169.0,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,A_DECLINED,0 days 09:36:20.319000


In [42]:
c=0
s=df.iloc[1,9]
time_till_next={}
for j in list_of_events:
    for i in range (1,244189):
        if df.iloc[i,3]==j: 
            a=s+(df.iloc[i+1]["case:REG_DATE"]-df.iloc[i]["case:REG_DATE"])
            c=c+1
    time_till_next[j] = s/c
time_till_next

{'A_SUBMITTED': Timedelta('0 days 00:00:00.000025523'),
 'A_PARTLYSUBMITTED': Timedelta('0 days 00:00:00.000012761'),
 'A_PREACCEPTED': Timedelta('0 days 00:00:00.000009958'),
 'W_Completeren aanvraag': Timedelta('0 days 00:00:00.000004077'),
 'A_ACCEPTED': Timedelta('0 days 00:00:00.000003837'),
 'O_SELECTED': Timedelta('0 days 00:00:00.000003551'),
 'A_FINALIZED': Timedelta('0 days 00:00:00.000003371'),
 'O_CREATED': Timedelta('0 days 00:00:00.000003147'),
 'O_SENT': Timedelta('0 days 00:00:00.000002952'),
 'W_Nabellen offertes': Timedelta('0 days 00:00:00.000002116'),
 'O_SENT_BACK': Timedelta('0 days 00:00:00.000002070'),
 'W_Valideren aanvraag': Timedelta('0 days 00:00:00.000001840'),
 'A_REGISTERED': Timedelta('0 days 00:00:00.000001817'),
 'A_APPROVED': Timedelta('0 days 00:00:00.000001796'),
 'O_ACCEPTED': Timedelta('0 days 00:00:00.000001774'),
 'A_ACTIVATED': Timedelta('0 days 00:00:00.000001753'),
 'O_CANCELLED': Timedelta('0 days 00:00:00.000001720'),
 'W_Wijzigen contractg

In [43]:
df['time_till_next'] = df['concept:name'].map(time_till_next)

<ipython-input-43-16971623ba36>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_till_next'] = df['concept:name'].map(time_till_next)


In [44]:
df

,Unnamed: 0,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,next_common_event,time_till_next
0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_SUBMITTED,0 days 00:00:00.000025523
1,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PARTLYSUBMITTED,0 days 00:00:00.000012761
2,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PREACCEPTED,0 days 00:00:00.000009958
3,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,W_Completeren aanvraag,0 days 00:00:00.000004077
5,5,10862.0,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_ACCEPTED,0 days 00:00:00.000003837
...,...,...,...,...,...,...,...,...,...,...
262195,262195,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,A_PARTLYSUBMITTED,0 days 00:00:00.000012761
262196,262196,112.0,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads,0 days 00:00:00.000001521
262197,262197,11169.0,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads,0 days 00:00:00.000001521
262198,262198,11169.0,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,A_DECLINED,0 days 00:00:00.000001655


In [45]:
df['next_common_event'] = df['concept:name'].map(loaded_dict)


<ipython-input-45-4f1646baaaf0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['next_common_event'] = df['concept:name'].map(loaded_dict)


In [46]:
df

,Unnamed: 0,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,next_common_event,time_till_next
0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PARTLYSUBMITTED,0 days 00:00:00.000025523
1,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,A_PREACCEPTED,0 days 00:00:00.000012761
2,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,W_Completeren aanvraag,0 days 00:00:00.000009958
3,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,W_Completeren aanvraag,0 days 00:00:00.000004077
5,5,10862.0,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000,O_SELECTED,0 days 00:00:00.000003837
...,...,...,...,...,...,...,...,...,...,...
262195,262195,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,A_PREACCEPTED,0 days 00:00:00.000012761
262196,262196,112.0,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads,0 days 00:00:00.000001521
262197,262197,11169.0,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads,0 days 00:00:00.000001521
262198,262198,11169.0,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000,W_Afhandelen leads,0 days 00:00:00.000001655
